In [2]:
import pandas as pd
from pathlib import Path

BASE = Path("./ml-25m/ml-25m")  # 로컬 압축해제 경로

# ---------- 1) CSV 로드 (필요 컬럼만) ----------
ratings = pd.read_csv(
    BASE/"ratings.csv",
    usecols=["userId","movieId","rating","timestamp"]
)
movies = pd.read_csv(
    BASE/"movies.csv",
    usecols=["movieId","title","genres"]
)
links = pd.read_csv(
    BASE/"links.csv",
    usecols=["movieId","imdbId","tmdbId"]
)
tags = pd.read_csv(
    BASE/"tags.csv",
    usecols=["userId","movieId","tag","timestamp"]  # tag의 timestamp는 나중에 안 쓸 예정
)

# ---------- 2) timestamp -> datetime (ratings 기준) ----------
ratings["datetime"] = pd.to_datetime(ratings["timestamp"], unit="s")

# ---------- 3) tags 집계 (genome-scores-0.05 기반으로 필터링 후 집계) ----------
# 3-1) 기본 정리
tags_clean = tags.dropna(subset=["tag"]).copy()

def norm_tag(s):
    return str(s).strip().lower()

tags_clean["tag_norm"] = tags_clean["tag"].apply(norm_tag)

# 3-2) genome 기반 허용 태그 테이블 생성
g_tags   = pd.read_csv(BASE/"genome-tags.csv", usecols=["tagId","tag"])
g_scores = pd.read_csv(BASE/"genome-scores-0.5.csv", usecols=["movieId","tagId","relevance"])

g_tags["tag_norm"] = g_tags["tag"].apply(norm_tag)

# relevance > 0.05인 (movieId, tag_norm) 조합만 추출
allowed = (
    g_scores.merge(g_tags[["tagId","tag_norm"]], on="tagId", how="left")
            [["movieId","tag_norm"]]
            .dropna()
            .drop_duplicates()
)

# 3-3) 사용자 태그(tags.csv)를 허용 목록으로 필터링
tags_allowed = (
    tags_clean.merge(allowed, on=["movieId","tag_norm"], how="inner")
)

# 3-4) 영화별로 태그 집계 (원래 문자열 보존, 중복 제거, 정렬 후 ' | '로 연결)
tags_agg = (
    tags_allowed
    .groupby(["movieId"], as_index=False)["tag"]
    .agg(lambda s: " | ".join(sorted(set(t.strip() for t in s if str(t).strip()))))
    .rename(columns={"tag":"movie_tags"})
)

# ---------- 4) 순차 머지 ----------
# ratings ⟶ movies (movieId)
df = ratings.merge(movies, on="movieId", how="left")

# (선택) tmdbId 정수 캐스팅을 원한다면: 결측 허용 정수형으로
# df["tmdbId"]는 아직 없으므로 links를 머지한 뒤 처리
# ratings+movies ⟶ links (movieId)
df = df.merge(links, on="movieId", how="left")

# imdbId, tmdbId 보조 URL 컬럼(선택)
df["imdb_url"] = "https://www.imdb.com/title/tt" + df["imdbId"].astype("Int64").astype(str) + "/"
# tmdbId가 NaN이면 URL 만들지 않음
df["tmdb_url"] = df["tmdbId"].apply(lambda x: f"https://www.themoviedb.org/movie/{int(x)}" if pd.notna(x) else pd.NA)

# ratings+movies+links ⟶ tags 집계 (userId+movieId)
# df = df.merge(tags_agg, on=["userId","movieId"], how="left")
df = df.merge(tags_agg, on=["movieId"], how="left")

# (선택) tmdbId를 결측 허용 정수로 캐스팅
df["tmdbId"] = df["tmdbId"].astype("Int64")
df["imdbId"] = df["imdbId"].astype("Int64")

# ---------- 5) 결과 확인 ----------
print(df.shape)
df.head(10)

(25000095, 12)


,userId,movieId,rating,timestamp,datetime,title,genres,imdbId,tmdbId,imdb_url,tmdb_url,movie_tags
0,1,296,5.0,1147880044,2006-05-17 15:34:04,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,110912,680,https://www.imdb.com/title/tt110912/,https://www.themoviedb.org/movie/680,AWESOME | Black comedy | Crime | Dialogue | Go...
1,1,306,3.5,1147868817,2006-05-17 12:26:57,Three Colors: Red (Trois couleurs: Rouge) (1994),Drama,111495,110,https://www.imdb.com/title/tt111495/,https://www.themoviedb.org/movie/110,Criterion | French | atmospheric | cinematogra...
2,1,307,5.0,1147868828,2006-05-17 12:27:08,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama,108394,108,https://www.imdb.com/title/tt108394/,https://www.themoviedb.org/movie/108,Criterion | French | atmospheric | classical m...
3,1,665,5.0,1147878820,2006-05-17 15:13:40,Underground (1995),Comedy|Drama|War,114787,11902,https://www.imdb.com/title/tt114787/,https://www.themoviedb.org/movie/11902,Golden Palm | World War II | cerebral | crazy ...
4,1,899,3.5,1147868510,2006-05-17 12:21:50,Singin' in the Rain (1952),Comedy|Musical|Romance,45152,872,https://www.imdb.com/title/tt45152/,https://www.themoviedb.org/movie/872,1920s | AFI 100 | AFI 100 (Laughs) | Dance | H...
5,1,1088,4.0,1147868495,2006-05-17 12:21:35,Dirty Dancing (1987),Drama|Musical|Romance,92890,88,https://www.imdb.com/title/tt92890/,https://www.themoviedb.org/movie/88,1980s | Coming of Age | Dance | Dancing | Fami...
6,1,1175,3.5,1147868826,2006-05-17 12:27:06,Delicatessen (1991),Comedy|Drama|Romance,101700,892,https://www.imdb.com/title/tt101700/,https://www.themoviedb.org/movie/892,Black comedy | French | Post apocalyptic | alt...
7,1,1217,3.5,1147878326,2006-05-17 15:05:26,Ran (1985),Drama|War,89881,11645,https://www.imdb.com/title/tt89881/,https://www.themoviedb.org/movie/11645,Criterion | Japan | Kurosawa | Shakespeare | T...
8,1,1237,5.0,1147868839,2006-05-17 12:27:19,"Seventh Seal, The (Sjunde inseglet, Det) (1957)",Drama,50976,490,https://www.imdb.com/title/tt50976/,https://www.themoviedb.org/movie/490,Breathtaking | Criterion | Death | Swedish | a...
9,1,1250,4.0,1147868414,2006-05-17 12:20:14,"Bridge on the River Kwai, The (1957)",Adventure|Drama|War,50212,826,https://www.imdb.com/title/tt50212/,https://www.themoviedb.org/movie/826,AFI 100 | Best War Films | British | Oscar (Be...


In [11]:
total = len(df)
tagged = df['movie_tags'].notna().sum()
na_count = df['movie_tags'].isna().sum()

print(f"전체 행 수: {total:,}")
print(f"태그가 있는 행 수: {tagged:,}")
print(f"태그가 없는 행 수: {na_count:,}")
print(f"태그가 있는 비율: {tagged / total * 100:.2f}%")


전체 행 수: 25,000,095
태그가 있는 행 수: 24,440,235
태그가 없는 행 수: 559,860
태그가 있는 비율: 97.76%


In [12]:
df.isna().sum()

userId             0
movieId            0
rating             0
timestamp          0
datetime           0
title              0
genres             0
imdbId             0
tmdbId          2431
imdb_url           0
tmdb_url        2431
movie_tags    559860
dtype: int64

In [13]:
max_count = (
    df["movie_tags"]
      .fillna("")
      .apply(lambda s: sum(1 for t in s.split(" | ") if t.strip()))
      .max()
)
print("row 단위 movie_tags 최대 개수:", max_count)


row 단위 movie_tags 최대 개수: 119


In [3]:
# ---------- 6) 장르 유니크값 확인 ----------
# 1) 각 영화의 장르를 "|"로 분리 후 전체 집합 생성
unique_genres = set()
for g in movies["genres"].dropna():
    unique_genres.update(g.split("|"))

print(f"🎬 장르 개수: {len(unique_genres)}개")
print("🎭 장르 목록:", sorted(unique_genres))


🎬 장르 개수: 20개
🎭 장르 목록: ['(no genres listed)', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']


In [4]:
# ---------- A) movie_tags -> tagId 매핑을 위한 준비 ----------
# 1) df에서 movieId, movie_tags만 추출
movie_tags_df = df[["movieId", "movie_tags"]].drop_duplicates().copy()

# 2) 태그 분리/정규화 (explode)
def split_and_norm(s):
    if pd.isna(s):
        return []
    return [t.strip() for t in str(s).split("|") if str(t).strip()]

movie_tags_long = (
    movie_tags_df
    .assign(tag_raw_list=lambda x: x["movie_tags"].apply(split_and_norm))
    .explode("tag_raw_list", ignore_index=True)
)

# 결측 처리 (movie_tags가 없는 영화는 제외)
movie_tags_long = movie_tags_long.dropna(subset=["tag_raw_list"]).copy()

# 3) 정규화 키 생성 (소문자 + strip)
movie_tags_long["tag_norm"] = movie_tags_long["tag_raw_list"].str.strip().str.lower()

# 4) genome-tags의 정규화 키와 매핑하여 tagId 부여
g_tags_norm = g_tags.copy()
g_tags_norm["tag_norm"] = g_tags_norm["tag"].str.strip().str.lower()

movie_tagid_long = (
    movie_tags_long
    .merge(g_tags_norm[["tagId","tag_norm"]], on="tag_norm", how="inner")
    [["movieId","tag_raw_list","tagId"]]
    .drop_duplicates()
)

# ---------- B) (movieId, tagId) -> relevance 조인 ----------
# g_scores: (movieId, tagId, relevance)
movie_tagid_score_long = (
    movie_tagid_long
    .merge(g_scores, on=["movieId","tagId"], how="left")
)

# ---------- C) 같은 영화 안에서 tagId와 relevance를 '|'로 이어 붙이기 ----------
def agg_join_sorted(df_grp):
    # tagId 기준 오름차순 정렬 후, tagId와 relevance를 같은 순서로 결합
    d = df_grp.sort_values("tagId")
    tag_ids = d["tagId"].astype("Int64").astype(str).tolist()
    # relevance가 없는 경우는 빈 문자열 대신 NA로 두고, 조인에서 제외하지 않음
    rels = d["relevance"].apply(lambda x: ("" if pd.isna(x) else f"{x:.5f}")).tolist()
    return pd.Series({
        "tagIds_str": "|".join(tag_ids) if len(tag_ids) else pd.NA,
        "relevance_str": "|".join(rels) if len(rels) else pd.NA,
    })

movie_tagid_score_agg = (
    movie_tagid_score_long
    .groupby("movieId", as_index=False)
    .apply(agg_join_sorted)
    .reset_index(drop=True)
)

# ---------- D) 원래 df에 합치기 ----------
df_with_tag = df.merge(movie_tagid_score_agg, on="movieId", how="left")

# 결과 확인
df_with_tag[["movieId","movie_tags","tagIds_str","relevance_str"]].head(10)


,movieId,movie_tags,tagIds_str,relevance_str
0,296,AWESOME | Black comedy | Crime | Dialogue | Go...,19|80|81|86|93|93|141|141|145|146|169|212|215|...,0.51425|0.66525|0.72050|0.72950|0.63150|0.6315...
1,306,Criterion | French | atmospheric | cinematogra...,86|212|270|316|351|402|411|569|618|639|641|845,0.62525|0.84500|0.93875|0.71625|0.98475|0.9570...
2,307,Criterion | French | atmospheric | classical m...,86|217|270|292|351|402|411|613|618|639|641|684...,0.65275|0.80725|0.96550|0.85575|0.98775|0.9155...
3,665,Golden Palm | World War II | cerebral | crazy ...,192|265|282|402|443|470|493|611|995|1121|1121,0.72500|0.50150|0.53900|0.79875|0.98925|0.5675...
4,899,1920s | AFI 100 | AFI 100 (Laughs) | Dance | H...,4|31|32|140|166|215|230|283|283|284|414|514|51...,0.54100|0.80175|0.60975|0.65250|0.83150|0.8942...
5,1088,1980s | Coming of Age | Dance | Dancing | Fami...,9|198|201|235|235|278|283|283|284|284|374|439|...,0.72475|0.64475|0.96600|0.77200|0.77200|0.5250...
6,1175,Black comedy | French | Post apocalyptic | alt...,52|86|141|141|180|212|218|223|230|276|285|286|...,0.78425|0.83400|0.88175|0.88175|0.81900|0.9112...
7,1217,Criterion | Japan | Kurosawa | Shakespeare | T...,82|86|109|120|145|167|212|215|270|348|359|401|...,0.71100|0.72275|0.61750|0.70700|0.62150|0.8045...
8,1237,Breathtaking | Criterion | Death | Swedish | a...,35|46|78|86|140|162|192|199|212|215|270|292|29...,0.91500|0.96025|0.54975|0.73075|0.88075|0.8080...
9,1250,AFI 100 | Best War Films | British | Oscar (Be...,22|31|107|127|164|164|215|359|536|575|576|584|...,0.89275|0.79750|0.67000|0.99225|0.89675|0.8967...


In [7]:
# 0) 개수 일치 여부 계산 (movie_tags vs tagIds_str vs relevance_str)
def count_pipe_items(s, sep="|", drop_blank=True):
    if pd.isna(s):
        return 0
    parts = [p.strip() for p in str(s).split(sep)]
    if drop_blank:
        parts = [p for p in parts if p != ""]
    return len(parts)

df_with_tag = df_with_tag.copy()
df_with_tag["n_movie_tags"] = df_with_tag["movie_tags"].apply(count_pipe_items)
df_with_tag["n_tagIds"]     = df_with_tag["tagIds_str"].apply(count_pipe_items)
df_with_tag["n_rels"]       = df_with_tag["relevance_str"].apply(count_pipe_items)

# 1) (요청 2번) movie_tags 갯수, relevance 갯수 안 맞는 행 삭제
#    tagIds_str(=태그 개수)와 relevance_str(=relevance 개수)가 movie_tags 개수와 모두 동일하고 >0 인 행만 유지
df_with_tag = df_with_tag.query("n_movie_tags == n_tagIds == n_rels and n_movie_tags > 0").copy()

# 2) (요청 3,4번) timestamp, imdbId, tmdbId 제거
df_with_tag.drop(columns=["timestamp","imdbId","tmdbId"], errors="ignore", inplace=True)

# 3) 최종 스펙에 맞춰 컬럼명 정리
#    movie_tags -> tag, tagIds_str -> tagId, relevance_str -> relevance
df_final = df_with_tag.rename(columns={
    "movie_tags":   "tag",
    "tagIds_str":   "tagId",
    "relevance_str":"relevance",
})

# 4) (요청 1번) 핵심 컬럼에 NaN 존재하는 행 삭제
need_cols = ["userId","movieId","rating","datetime","title","genres",
             "imdb_url","tmdb_url","tag","tagId","relevance"]
df_final = df_final.dropna(subset=need_cols).copy()

# 5) 최종 컬럼 순서로 선택
df_final = df_final[need_cols]

# 6) 저장
out_path = "./movie.csv"
df_final.to_csv(out_path, index=False)
print(f"Saved: {out_path}  rows={len(df_final)}  cols={len(df_final.columns)}")

Saved: ./movie.csv  rows=24438721  cols=11


In [1]:
import pandas as pd

# CSV 파일 읽기
df = pd.read_csv('./movie.csv')

# tag와 relevance 개수 불일치 확인
mismatched = []

for idx, row in df.iterrows():
    # tag와 relevance를 '|'로 분리
    tags = str(row['tag']).split('|') if pd.notna(row['tag']) else []
    relevances = str(row['relevance']).split('|') if pd.notna(row['relevance']) else []
    
    # 개수 비교
    if len(tags) != len(relevances):
        mismatched.append({
            'index': idx,
            'movieId': row.get('movieId', 'N/A'),
            'tag_count': len(tags),
            'relevance_count': len(relevances),
            'difference': len(tags) - len(relevances)
        })

# 결과 출력
if mismatched:
    print(f"⚠️  총 {len(mismatched)}개의 불일치 발견!\n")
    mismatch_df = pd.DataFrame(mismatched)
    print(mismatch_df.to_string(index=False))
    
    # 통계 정보
    print(f"\n📊 통계:")
    print(f"전체 행 수: {len(df)}")
    print(f"불일치 행 수: {len(mismatched)}")
    print(f"불일치 비율: {len(mismatched)/len(df)*100:.2f}%")
else:
    print("✅ 모든 행에서 tag와 relevance 개수가 일치합니다!")
    print(f"전체 {len(df)}개 행 확인 완료")

✅ 모든 행에서 tag와 relevance 개수가 일치합니다!
전체 24438721개 행 확인 완료


In [3]:
# 6) NaN 존재 여부 확인 및 제거
na_counts = df.isna().sum()
print("NaN counts per column:\n", na_counts)

NaN counts per column:
 userId       0
movieId      0
rating       0
datetime     0
title        0
genres       0
imdb_url     0
tmdb_url     0
tag          0
tagId        0
relevance    0
dtype: int64


In [2]:
import re
import pandas as pd

SRC = "./movie.csv"
DST = "./movie_tot.csv"

# 1) 로드
df = pd.read_csv(SRC)

# 2) imdbId_url 정리 (컬럼명이 imdb_url이면 복사)
if "imdbId_url" not in df.columns and "imdb_url" in df.columns:
    df["imdbId_url"] = df["imdb_url"]

# 3) title에서 years 추출 + title 정제
def split_title_year(s: str):
    if pd.isna(s):
        return s, pd.NA
    m = re.search(r"\((\d{4})\)\s*$", s)
    if m:
        year = int(m.group(1))
        title_clean = re.sub(r"\s*\(\d{4}\)\s*$", "", s).strip()
        return title_clean, year
    return s.strip(), pd.NA

title_year = df["title"].apply(split_title_year)
df["title"] = title_year.apply(lambda x: x[0])
df["years"] = title_year.apply(lambda x: x[1])

# 4) tags 앞에 genres 추가, relevance 앞에 genres 개수만큼 1.0 추가
def prepend_genres_with_scores(row):
    """genres를 tag 앞에 추가하고, genres 개수만큼 1.0을 relevance 앞에 추가"""
    genres = "" if pd.isna(row.get("genres", "")) else str(row.get("genres", "")).strip()
    tag = "" if pd.isna(row.get("tag", "")) else str(row.get("tag", "")).strip()
    relevance = "" if pd.isna(row.get("relevance", "")) else str(row.get("relevance", "")).strip()
    
    if genres == "":
        # genres가 없으면 원본 그대로
        return tag, relevance
    
    # genres를 '|'로 분리하여 개수 계산
    genre_list = [g.strip() for g in genres.split("|") if g.strip()]
    genre_count = len(genre_list)
    
    # tag: genres를 맨 앞에 추가
    new_tag = genres if tag == "" else f"{genres}|{tag}"
    
    # relevance: genres 개수만큼 1.0 추가
    scores = "|".join(["1.0"] * genre_count)
    new_relevance = scores if relevance == "" else f"{scores}|{relevance}"
    
    return new_tag, new_relevance

# 적용
result = df.apply(prepend_genres_with_scores, axis=1)
df["tag"] = result.apply(lambda x: x[0])
df["relevance"] = result.apply(lambda x: x[1])

# 5) 검증: tag와 relevance 개수 확인
print("🔍 검증 중...")
mismatched = []
for idx, row in df.iterrows():
    tags = str(row['tag']).split('|') if pd.notna(row['tag']) and str(row['tag']).strip() else []
    relevances = str(row['relevance']).split('|') if pd.notna(row['relevance']) and str(row['relevance']).strip() else []
    
    if len(tags) != len(relevances):
        mismatched.append({
            'index': idx,
            'tag_count': len(tags),
            'relevance_count': len(relevances),
            'difference': len(tags) - len(relevances)
        })
    
    # 진행 상황 출력 (100만 행마다)
    if (idx + 1) % 1000000 == 0:
        print(f"  검증 중... {idx + 1}/{len(df)} 행")

if mismatched:
    print(f"\n⚠️  {len(mismatched)}개의 불일치 발견!")
    print(pd.DataFrame(mismatched).head(20))
else:
    print("\n✅ 모든 행에서 tag와 relevance 개수가 일치합니다!")

# 6) 최종 컬럼만 선택 및 저장
final_cols = ["userId", "movieId", "years", "rating", "title", "imdbId_url", "tag", "relevance"]
df_final = df.loc[:, [c for c in final_cols if c in df.columns]]

print(f"\n📋 결과 샘플 (처음 10개 행):")
print(df_final.head(10))

# 저장
df_final.to_csv(DST, index=False)
print(f"\n💾 Saved: {DST}")

🔍 검증 중...
  검증 중... 1000000/24438721 행
  검증 중... 2000000/24438721 행
  검증 중... 3000000/24438721 행
  검증 중... 4000000/24438721 행
  검증 중... 5000000/24438721 행
  검증 중... 6000000/24438721 행
  검증 중... 7000000/24438721 행
  검증 중... 8000000/24438721 행
  검증 중... 9000000/24438721 행
  검증 중... 10000000/24438721 행
  검증 중... 11000000/24438721 행
  검증 중... 12000000/24438721 행
  검증 중... 13000000/24438721 행
  검증 중... 14000000/24438721 행
  검증 중... 15000000/24438721 행
  검증 중... 16000000/24438721 행
  검증 중... 17000000/24438721 행
  검증 중... 18000000/24438721 행
  검증 중... 19000000/24438721 행
  검증 중... 20000000/24438721 행
  검증 중... 21000000/24438721 행
  검증 중... 22000000/24438721 행
  검증 중... 23000000/24438721 행
  검증 중... 24000000/24438721 행

✅ 모든 행에서 tag와 relevance 개수가 일치합니다!

📋 결과 샘플 (처음 10개 행):
   userId  movieId years  rating                                      title  \
0       1      296  1994     5.0                               Pulp Fiction   
1       1      306  1994     3.5  Three Colors: Red (Trois coule

In [3]:
import pandas as pd

# CSV 파일 읽기
df = pd.read_csv('./movie_tot.csv')

# tag와 relevance 개수 불일치 확인
mismatched = []

for idx, row in df.iterrows():
    # tag와 relevance를 '|'로 분리
    tags = str(row['tag']).split('|') if pd.notna(row['tag']) else []
    relevances = str(row['relevance']).split('|') if pd.notna(row['relevance']) else []
    
    # 개수 비교
    if len(tags) != len(relevances):
        mismatched.append({
            'index': idx,
            'movieId': row.get('movieId', 'N/A'),
            'tag_count': len(tags),
            'relevance_count': len(relevances),
            'difference': len(tags) - len(relevances)
        })

# 결과 출력
if mismatched:
    print(f"⚠️  총 {len(mismatched)}개의 불일치 발견!\n")
    mismatch_df = pd.DataFrame(mismatched)
    print(mismatch_df.to_string(index=False))
    
    # 통계 정보
    print(f"\n📊 통계:")
    print(f"전체 행 수: {len(df)}")
    print(f"불일치 행 수: {len(mismatched)}")
    print(f"불일치 비율: {len(mismatched)/len(df)*100:.2f}%")
else:
    print("✅ 모든 행에서 tag와 relevance 개수가 일치합니다!")
    print(f"전체 {len(df)}개 행 확인 완료")

✅ 모든 행에서 tag와 relevance 개수가 일치합니다!
전체 24438721개 행 확인 완료


In [4]:
na_counts = df.isna().sum()
print("NaN counts per column:\n", na_counts)

NaN counts per column:
 userId           0
movieId          0
years         9008
rating           0
title            0
imdbId_url       0
tag              0
relevance        0
dtype: int64


In [5]:
# 6) NaN 존재 여부 확인 및 제거
na_counts = df.isna().sum()
print("NaN counts per column:\n", na_counts)

# 하나라도 NaN 있으면 그 행 제거
out_clean = df_final.dropna(how="any").reset_index(drop=True)
dropped = len(df_final) - len(out_clean)
print(f"Rows dropped due to NaN: {dropped}")

# 7) 최종본 저장
df_final.to_csv(DST, index=False)
print(f"Saved: {DST}")

NaN counts per column:
 userId           0
movieId          0
years         9008
rating           0
title            0
imdbId_url       0
tag              0
relevance        0
dtype: int64
Rows dropped due to NaN: 9008
Saved: ./movie_tot.csv


In [5]:
import pandas as pd

# CSV 파일 읽기
#df = pd.read_csv('./movie.csv')

# tag와 relevance 개수 불일치 확인
mismatched = []

for idx, row in df_final.iterrows():
    # tag와 relevance를 '|'로 분리
    tags = str(row['tag']).split('|') if pd.notna(row['tag']) else []
    relevances = str(row['relevance']).split('|') if pd.notna(row['relevance']) else []
    
    # 개수 비교
    if len(tags) != len(relevances):
        mismatched.append({
            'index': idx,
            'movieId': row.get('movieId', 'N/A'),
            'tag_count': len(tags),
            'relevance_count': len(relevances),
            'difference': len(tags) - len(relevances)
        })

# 결과 출력
if mismatched:
    print(f"⚠️  총 {len(mismatched)}개의 불일치 발견!\n")
    mismatch_df = pd.DataFrame(mismatched)
    print(mismatch_df.to_string(index=False))
    
    # 통계 정보
    print(f"\n📊 통계:")
    print(f"전체 행 수: {len(df)}")
    print(f"불일치 행 수: {len(mismatched)}")
    print(f"불일치 비율: {len(mismatched)/len(df)*100:.2f}%")
else:
    print("✅ 모든 행에서 tag와 relevance 개수가 일치합니다!")
    print(f"전체 {len(df)}개 행 확인 완료")

⚠️  총 20577055개의 불일치 발견!

